In [2]:
%pip install pygad

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pygad

### Prepare target flow curve

In [11]:
# Read the CSV file into a DataFrame
df = pd.read_csv('abaqus simulation F-D fitting/Flowcurve_RT.csv')
# print(df)
# Extract the true strain and average true stress columns
trueStrain = df['True strain']
trueStress = df['Avg.True stress']

In [25]:
# Discrete searching space
param_bounds = {
    "c1": {'low': 500, 'high': 2000, 'step': 0.1},  
    "c2": {'low': 0.1, 'high': 10, 'step': 0.001},    
    "c3": {'low': 0.01, 'high': 0.1, 'step': 0.0001}  
}

# Continuous searching space
param_bounds = {
    "c1": {'low': 500, 'high': 2000},  
    "c2": {'low': 0.1, 'high': 10},    
    "c3": {'low': 0.01, 'high': 0.1}  
}

# Swift Laws
def swift_law(c1,c2,c3,strain):
    stress=c1*(c2+strain)**c3 #3 parameter 
    return(stress)

# Note: GA in PyGAD tries to maximize, so you should use the inverse of the loss function.
def lossFunction(GA_instance, solution, solution_idx):
    c1 = solution[0]
    c2 = solution[1]
    c3 = solution[2]
    simStress = swift_law(c1,c2,c3,trueStrain)
    # RMSE loss
    fitness = np.sqrt(np.mean((simStress - trueStress)**2))
    loss = 1.0 / fitness
    return loss

In [26]:
class GA():

    ##################################
    # OPTIMIZER CLASS INITIALIZATION #
    ##################################

    def __init__(self):    
        #############################
        # Optimizer hyperparameters #
        #############################

        # You can modify these settings. 
        # Documentations:
        # https://pygad.readthedocs.io/en/latest/README_pygad_ReadTheDocs.html#pygad-ga-class
        self.num_generations=100
        self.num_parents_mating=200
        self.sol_per_pop=1000
        self.allow_duplicate_genes=False
        self.parent_selection_type="sss"
        self.crossover_type="single_point"
        self.mutation_type="random"
        self.mutation_num_genes=1
        self.keep_elitism=200
        self.stop_criteria=["reach_0.5", "saturate_15"]

    ##########################
    # OPTIMIZATION FUNCTIONS #
    ##########################

    def initializeOptimizer(self, lossFunction, param_bounds):
        self.param_bounds = param_bounds
        GA_bounds = []
        for param in self.param_bounds:
            GA_bounds.append(param_bounds[param])
        # print(GA_bounds)
        num_genes = len(GA_bounds)        
        ga_instance = pygad.GA(fitness_func=lossFunction,
                            num_generations=self.num_generations, 
                            num_parents_mating=self.num_parents_mating, 
                            sol_per_pop=self.sol_per_pop, 
                            num_genes=num_genes,
                            gene_space=GA_bounds,
                            allow_duplicate_genes=self.allow_duplicate_genes,
                            parent_selection_type=self.parent_selection_type,
                            crossover_type=self.crossover_type,
                            mutation_type=self.mutation_type,
                            keep_elitism=self.keep_elitism,
                            mutation_num_genes=self.mutation_num_genes,
                            stop_criteria=self.stop_criteria,
                            parallel_processing=["thread", 5],
                            )
        self.optimizer = ga_instance

    def run(self):
        self.optimizer.run()

    def outputResult(self):
        best_solution, best_solution_loss, best_solution_index = self.optimizer.best_solution()
        solution_dict = {}
        for i, param in enumerate(self.param_bounds):
            solution_dict[param] = best_solution[i]        
        solution_tuple = tuple(solution_dict.items())
        return solution_dict, solution_tuple, best_solution_loss

In [27]:
GA_instance = GA()
GA_instance.initializeOptimizer(lossFunction, param_bounds)
GA_instance.run()
solution_dict, solution_tuple, best_solution_loss = GA_instance.outputResult()

for param in solution_dict:
    print(f"{param}: {solution_dict[param]}")

print(f"Best solution loss = {best_solution_loss:.4f}")

[{'low': 500, 'high': 2000}, {'low': 0.1, 'high': 10}, {'low': 0.01, 'high': 0.1}]
c1: 1190.8481543758592
c2: 0.10044562043570893
c3: 0.09983790950185621
